In [1]:
random_seed = 42

import torch
from torch import nn
import torch.nn.functional as F
import random
import numpy as np
import os
random.seed(random_seed)

np.random.seed(random_seed)
torch.manual_seed(random_seed)
# Включите куду
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
os.environ['PYTHONHASHSEED'] = str(random_seed)

In [2]:
#! wget https://raw.githubusercontent.com/DLSchool/dlschool_old/master/materials/homeworks/hw09/dostoevsky.txt

## char-RNN in PyTorch

Тут нам предстоит построить character-level RNN c помощью PyTroch. Для того чтобы освежить свои знания о данной модели рекомендую к прочтению [сию статью](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) написанную Андреем Карпатым.  Сразу проясню, тут мы будем 

In [3]:
import codecs
import io
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Обновите текст, если что-то потерлось. Можно сделать это не только в памяти компьютера, но и в своей собственной.

In [4]:
with io.open('dostoevsky.txt', 'r',encoding='utf8') as f:
    text = f.read().replace(u'\xa0', u' ').replace(u'\ufeff','')

In [5]:
text[:300]

'Федор Михайлович Достоевский\nБедные люди\nОх уж эти мне сказочники! Нет чтобы написать что-нибудь полезное, приятное, усладительное, а то всю подноготную в земле вырывают!.. Вот уж запретил бы им писать! Ну, на что это похоже: читаешь… невольно задумываешься, — а там всякая дребедень и пойдет в голов'

Закодируем наш текст в цифры, как мы и обсуждали ранее.

In [6]:
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}
encoded = np.array([char2int[ch] for ch in text])

In [7]:
encoded.shape # Наш словарь получился очень большой

(11321980,)

#### Обработка данных

Будем использовать для представления букв one-hot вектора. Напишите функцию для этого.

In [8]:
def one_hot_encode(arr, n_labels):
    
    # Инициализируем вектора 
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # заполним 1 в соответсвующем месте
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Приводим к нужному размеру
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

Попробуйте сделать функцию ниже, которая строит генератор мини-батчей. Каждая последовательность будет длины `n_steps`

In [9]:
def get_batches(arr, n_seqs, n_steps):
    """
    Создание генератора, возвращающего минибатчи размера (n_seqs x seq_len) Numpy
    """    
    batch_size = n_seqs * n_steps
    n_batches = len(arr)//batch_size
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size]
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+n_steps]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

### Построение charRNN модели

Ниже вам будет необходимо написать свою char-rnn по данному описанию. Как всегда основные рекомендации: сначала пишем определение слоев в  init, затем описываем их вызов в forward.


Кроме того тут есть важная функция predict. (в блоге Карпатого очень подробно все описано и дабы не заниматься копипастой, я направляю вас туда). Для начала попробуйте проверить учится ли ваша сеть и только потом заполняйте метод predict.

Почему даем именно модель над символами? Потому что это очень простая структура, которая может быть применена не только к текстам, но и к музыке. 

In [10]:
class CharRNN(nn.Module):
    def __init__(self, tokens, n_steps=100, n_hidden=256, n_layers=2,
                               drop_prob=0.1, lr=0.001, use_embeddings=False):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        self.use_embeddings = use_embeddings
        
        self.chars = tokens
        self.vocab_size = len(self.chars)
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        self.dropout = nn.Dropout(self.drop_prob)
        if self.use_embeddings:
            emb_size = 64
            self.embedder = nn.Embedding(self.vocab_size, emb_size)
            self.lstm = nn.GRU(emb_size, self.n_hidden, self.n_layers,
                               batch_first=True, dropout=self.drop_prob)
        else:
            self.lstm = nn.GRU(self.vocab_size, self.n_hidden, self.n_layers,
                               batch_first=True, dropout=self.drop_prob)
        self.fc = nn.Linear(n_hidden, self.vocab_size)
        
        self.init_weights()
        
    def forward(self, x, hc):
        ''' Forward pass through the network '''
        if self.use_embeddings:
            x = self.embedder(torch.argmax(x, dim=-1).long())
        x, h = self.lstm(x, hc)
#         x = self.dropout(x)
        x = self.fc(x)
        return x, h
    
    def predict(self, char, h=None, top_k=None):
        """        
            Returns the predicted character and the hidden state.
        """
        
        if h is None:
            h = self.init_hidden(1).to(device)
        
        x = np.array([[self.char2int[char]]])
        x = one_hot_encode(x, self.vocab_size)
        
        inputs = torch.from_numpy(x).to(device)
        out, h = self.forward(inputs, h)

        p = F.softmax(out, dim=-1)
        
        if top_k is None:
            top_ch = np.arange(len(self.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.cpu().numpy().squeeze()
        
        p = p.cpu().detach().numpy().squeeze()
        # Choose 1/k 
        char = np.random.choice(top_ch, p=p/p.sum())
        
        return self.int2char[char], h
    
    def init_weights(self):
        ''' Initialize weights for fully connected layer '''        
        # Set bias tensor to all zeros
        self.fc.bias.data.fill_(0)
        # FC weights as random uniform
        self.fc.weight.data.uniform_(-1, 1)
        
    def init_hidden(self, n_seqs):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x n_seqs x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, n_seqs, self.n_hidden).zero_()
        

Для проверки функционирования у нас есть функция `train` , которая позволит провести вам большое число экспериментов.

In [11]:
def train(net, data, epochs=10, n_seqs=10, n_steps=50, lr=0.0005, clip=3, val_frac=0.1, cuda=False, print_every=200):
    net.train()
#     opt = torch.optim.Adam(net.parameters(), lr=lr)
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        h = net.init_hidden(n_seqs)
        for x, y in get_batches(data, n_seqs, n_steps):
            counter += 1
            
            # Кодируем данные и отправлячем
            x = one_hot_encode(x, n_chars)
            inputs = torch.from_numpy(x).to(device)
            targets = torch.from_numpy(y).to(device)
            
            #  замените на .copy будет работать стабильнее
            h = torch.tensor(h.data, device=device)

            net.zero_grad()
            
            output, h = net.forward(inputs, h)
            
            loss = criterion(output.view(n_seqs*n_steps,-1), targets.view(n_seqs*n_steps))
            loss.backward()
            
            # clip grad norm может вам помочь
            nn.utils.clip_grad_norm_(net.parameters(), 1)

            opt.step()
            
            if counter % print_every == 0:
                net.eval()
                # Get validation loss
                val_h = net.init_hidden(n_seqs)
                val_losses = []
                for x, y in get_batches(val_data, n_seqs, n_steps):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    inputs = torch.from_numpy(x).to(device)
                    targets = torch.from_numpy(y).to(device)
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = val_h.clone().detach()
                    output, val_h = net.forward(inputs, val_h)
                    val_loss = criterion(output.view(n_seqs*n_steps,-1), targets.view(n_seqs*n_steps))
                    val_losses.append(val_loss.item())
                   
                # Попробуем валидироваться таким способом
                prime = 'Дом '
                top_k = 2
                chars = [ch for ch in prime]
                vh = None
                for ch in prime:
                    char, vh = net.predict(ch, vh, top_k=top_k)
                for ii in range(10):
                    char, vh = net.predict(chars[-1], vh, top_k=top_k)
                    chars.append(char)
                    
                #chars.append(char)
                print(''.join(chars))
                net.train()
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)),
                       "Validation Perplexity: {:.4f}".format(np.exp(np.mean(val_losses))))
    return net

## Время тренировки!

 
Теперь мы можем тренировать сеть. Сначала мы создадим саму сеть, с некоторыми заданными гиперпараметрами. Затем определите размеры мини-партий (количество последовательностей и количество шагов) и начните обучение. С функцией поезда мы можем установить количество эпох, скорость обучения и другие параметры. Кроме того, мы можем запустить обучение на графическом процессоре, установив `cuda = True`. Сейчас google дает всем бесплатно использовать gpu с помощью сервиса codelab.


![a](https://www.apmpodcasts.org/wp-content/uploads/2015/06/adventure-time.jpg)

In [12]:
if 'net' in locals():
    del net

In [13]:
num_hidden_units = 512
use_embeddings = True
net = CharRNN(chars, n_hidden=num_hidden_units, n_layers=2, use_embeddings=use_embeddings).to(device)

In [14]:
n_seqs, n_steps = 128, 100
print_every = 100
train(net, encoded, epochs=15, n_seqs=n_seqs, n_steps=n_steps, lr=0.0005, cuda=True, print_every=print_every)

/home/mikhail/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Дом всем него 
Epoch: 1/15... Step: 100... Loss: 2.2304... Val Loss: 2.1759 Validation Perplexity: 8.8098
Дом возможно б
Epoch: 1/15... Step: 200... Loss: 2.0138... Val Loss: 1.9891 Validation Perplexity: 7.3090
Дом никак он п
Epoch: 1/15... Step: 300... Loss: 1.8952... Val Loss: 1.8774 Validation Perplexity: 6.5366
Дом воспританн
Epoch: 1/15... Step: 400... Loss: 1.8167... Val Loss: 1.7967 Validation Perplexity: 6.0296
Дом возможно в
Epoch: 1/15... Step: 500... Loss: 1.7758... Val Loss: 1.7424 Validation Perplexity: 5.7108
Дом войдет, и 
Epoch: 1/15... Step: 600... Loss: 1.6853... Val Loss: 1.6869 Validation Perplexity: 5.4028
Дом полинение 
Epoch: 1/15... Step: 700... Loss: 1.6836... Val Loss: 1.6476 Validation Perplexity: 5.1947
Дом ней наше в
Epoch: 2/15... Step: 800... Loss: 1.6458... Val Loss: 1.6203 Validation Perplexity: 5.0547
Дом них на пол
Epoch: 2/15... Step: 900... Loss: 1.6180... Val Loss: 1.5917 Validation Perplexity: 4.9121
Дом полным про
Epoch: 2/15... Step: 1000... Lo

Дом сердцем бо
Epoch: 10/15... Step: 7800... Loss: 1.2722... Val Loss: 1.3012 Validation Perplexity: 3.6736
Дом известиями
Epoch: 10/15... Step: 7900... Loss: 1.1993... Val Loss: 1.2993 Validation Perplexity: 3.6667
Дом отца нашег
Epoch: 11/15... Step: 8000... Loss: 1.2194... Val Loss: 1.2996 Validation Perplexity: 3.6680
Дом последний 
Epoch: 11/15... Step: 8100... Loss: 1.2526... Val Loss: 1.2961 Validation Perplexity: 3.6551
Дом себе полож
Epoch: 11/15... Step: 8200... Loss: 1.2052... Val Loss: 1.2988 Validation Perplexity: 3.6648
Дом начинают с
Epoch: 11/15... Step: 8300... Loss: 1.2067... Val Loss: 1.2965 Validation Perplexity: 3.6564
Дом из них не 
Epoch: 11/15... Step: 8400... Loss: 1.2338... Val Loss: 1.2965 Validation Perplexity: 3.6566
Дом немец подо
Epoch: 11/15... Step: 8500... Loss: 1.2293... Val Loss: 1.2964 Validation Perplexity: 3.6561
Дом их принять
Epoch: 11/15... Step: 8600... Loss: 1.2207... Val Loss: 1.2921 Validation Perplexity: 3.6404
Дом и почти не
Epoch: 11/15.

CharRNN(
  (dropout): Dropout(p=0.1, inplace=False)
  (embedder): Embedding(175, 64)
  (lstm): GRU(64, 512, num_layers=2, batch_first=True, dropout=0.1)
  (fc): Linear(in_features=512, out_features=175, bias=True)
)

![train](https://i.pinimg.com/474x/0e/58/69/0e5869297852211e8447d6b09fa1f4f5.jpg)

**Вопрос 6**

Введите целую часть итоговой  перплексии ex. [142.37 ] = 142

3

### Загрузка модели 

Чтобы настроить гиперпараметры для получения максимальной производительности, вам понадобятся наблюдения за обучением и валидацией. Если ваша потеря обучения намного ниже, чем потеря проверки, вы перерабатываете. Увеличьте регуляризацию (больше выпадений) или используйте меньшую сеть. Если потери обучения и проверки близки, вы недофинансируете, чтобы увеличить размер сети.

После обучения мы сохраним модель, чтобы мы могли загрузить ее позже, если понадобится. Здесь сохраняются параметры, необходимые для создания той же архитектуры, гиперпараметров скрытого слоя и текстовых символов.

In [15]:
checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars,
               'use_embeddings': use_embeddings}
with open('./rnn.net', 'wb') as f:
    torch.save(checkpoint, f)

## Семплирование

Теперь, когда модель обучена, мы захотим попробовать ее. Чтобы заполучить текст, мы передаем символ и сеть прогнозируем следующий символ. Затем мы берем новый символ, передаем его обратно и получаем еще один. Просто продолжайте делать это, и вы создадите кучу текста!

### Top K 

Наши прогнозы основаны на категориальном распределении вероятностей по всем возможным признакам. Мы можем сделать выборку более разумным, но менее переменным, учитывая только некоторые вероятные символы $ K $. Это будет препятствовать тому, чтобы сеть давала нам абсолютно абсурдные символы, позволяя ей вводить некоторый шум и случайность в выбранный текст.

Как правило, вы хотите настроить сеть, чтобы создать скрытое состояние. В противном случае сеть начнет генерировать символы наугад.]

Из основного задания, попробуйте сгенерировать небольшой рассказик. Поиграйте с параметрами. Найдите собственный датасет( чтобы проверяющим было весело ) и попробуйте обучить нейронную сеть на нем. 

Не забывайте эксперементировать со структурой. У LSTM много аналогий, посмотрите как они справляются с данной задачей и запишите это в отчет. 

**Вопрос 7**

Введите результат модели. Помните, он должен быть осмысленным.


In [16]:
def sample(net, size, prime='The', top_k=None, cuda=False):
    net.eval()
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = None
    for ch in prime:
        char, h = net.predict(ch, h,top_k=top_k)
        
    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = net.predict(chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [17]:
net.eval()
print(sample(net, 70, prime='Путин', top_k=2, cuda=True))

Путина, и все в ней возросло и в том, что он все веруют, что в таком случае 


Посмотрим на семлы случайной модели.

In [18]:
print(sample(CharRNN(chars, n_hidden=num_hidden_units, n_layers=2).to(device), 
             20, prime='Путин ', top_k=5, cuda=True))

Путин JИУJZJJИ:Z:ОJZИ:JИJИО


Разница видна, значит мы все сделали правильно.

In [19]:
with open('./rnn.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'], use_embeddings=checkpoint['use_embeddings'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

## Дополнительная часть


Тут предлагается описать все проделанные эксперименты. 

Попробуйте сделать его насыщенным необходимой информацией для проверяющих, чтобы можно было сразу оценить сколько было проделано работы. Пример вопросов для отчета был дан выше. Можно добавить семплы модели на эпохах 1, 3, 9, 15, для лучшего понимания обучения.

| hidden units   |use embeddings   | validation perplexity   |  
|---------|---|---|
| 32  | False   |6.6331 |
| 32  | True   |6.4652   |
| 64  | False   |5.3862 |
| 64  | True   | 5.1893  |
| 128  | False    | 4.5528 |
| 128  | True   | 4.4040 |
| 512  | True   | 3.6221|


Семплы модели на эпохах 1, 3, 9, 15

Epoch: 1
    
Дом полинение

Epoch: 3
    
Дом них на все

Epoch: 9
    
Дом сердцем св

Epoch: 15
    
Дом известных

В целом увеличение как количества ячеек так и добавление слоя эмбедингов позволяло снижать перплексию. 
При этом видно, что далекие зависимости сеть не улавливает и генерирует согласованные но лишенные смысла тексты